In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

def obtener_enlaces(url):
    # Realizar la petición HTTP a la página web
    respuesta = requests.get(url)
    # Si la petición fue exitosa, procesar el contenido
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.content, 'html.parser')
        # Extraer todos los elementos <a> de la página
        enlaces = soup.find_all('a', href=True)
        # Crear una lista para almacenar los datos
        datos = []
        # Recorrer cada enlace y almacenar la información necesaria
        for enlace in enlaces:
            # Extraer el href del enlace
            href = enlace['href']
            # Extraer el texto del enlace, que es el nombre del archivo
            texto = enlace.text
            # Intentar extraer las observaciones que están en el contenido antes del enlace
            # Esto asume que el texto de observaciones está justo antes del enlace
            observaciones = enlace.previous_sibling
            if observaciones:
                observaciones = observaciones.strip()  #Remueve los espacios al principio y al final de la línea
                # Dividir las observaciones en partes
                partes = observaciones.split("   ")
                #print(partes)
                # Extraer la fecha y la hora (primeras dos partes)
                fecha_hora = ' '.join(partes[:1])
                #print(fecha_hora)
                # Extraer el tamaño (última parte)
                tamaño = partes[-1]
                #print(tamaño)
            else:
                fecha_hora = "No disponible"
                tamaño = "No disponible"
            # Añadir a la lista como una tupla
            datos.append((texto, "http://rtv-b2b-api-logs.vigiloo.net" + href, fecha_hora, tamaño))    # datos.append((texto, url + href, fecha_hora, tamaño))
        # Convertir la lista en un DataFrame
        df = pd.DataFrame(datos, columns=['NombreArchivo', 'URL', 'FechaHora', 'Tamaño'])
        df.drop(0,axis=0)
        # Convertir la columna 'FechaHora' a tipo datetime
        df['FechaHora'] = pd.to_datetime(df['FechaHora'], dayfirst=False, format='%m/%d/%Y %I:%M %p',errors='coerce') 
        # Convertir la columna 'Tamaño' a tipo numérico
        df['Tamaño'] = pd.to_numeric(df['Tamaño'], errors='coerce')
        df['Embarque'] = df['NombreArchivo'].str.extract(r'-(\d+)\.log')
        df.drop(0,axis=0, inplace=True)  # Borro la Primera línea que apunta al directorio padre        
        return df
    else:
        return "Error al acceder a la página"

In [2]:
# URL de la página de donde se desean extraer los enlaces
url = "http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/"
archivos_df = obtener_enlaces(url)
if isinstance(archivos_df, pd.DataFrame):
   print(archivos_df.head(1))
else:
   print("Error")

            NombreArchivo                                                URL  \
1  638270843928515554.log  http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...   

            FechaHora  Tamaño Embarque  
1 2023-08-08 08:39:00  2404.0      NaN  


In [3]:
# FILTRAR Archivos_df y generar subset en archivo SALIDA

fecha_inicio = datetime.datetime(year=2024,month=5,day=1)

entrada = archivos_df[(archivos_df['FechaHora'] >=fecha_inicio)]
#salida.dropna()
# print(filtered_df)


In [4]:
entrada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 835 entries, 3282 to 4116
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   NombreArchivo  835 non-null    object        
 1   URL            835 non-null    object        
 2   FechaHora      835 non-null    datetime64[ns]
 3   Tamaño         835 non-null    float64       
 4   Embarque       834 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 39.1+ KB


In [5]:
# Extraer valores únicos de la columna 'Embarque'
embarques = entrada['Embarque'].unique()

# Formatear los valores entre apóstrofes y separados por comas
REGISTROS = ', '.join(f"'{embarque}'" for embarque in embarques)


In [6]:
print(REGISTROS)

'834246634', '834246635', '834246801', '834246802', '834246636', '834246995', '834247182', '834247392', '834247643', '834247810', '834247831', '834247887', '834247941', '834247974', '834247986', '834245202', '834245203', '834245204', '834248524', '834248525', '834248658', '834248657', '834245118', '834245161', '834248893', '834248894', '834245142', '834248856', '834245128', '834249678', '834249392', '834249379', '834249366', '834249367', '834249378', '834249374', '834249925', '40444', '80000804', '830009006', '830064274', '834205406', '834205445', '834214181', '834211973', '834205175', '834205303', '834250919', '834250918', '834251162', '834251163', '834251462', '834251463', '834251464', '834251466', '834251465', '834251467', '834251773', '834251774', '834251776', '834251780', '834252252', 'nan', '834245362', '834245388', '834252386', '834252335', '834252334', '834252367', '834252389', '834245386', '834245389', '834252347', '834252332', '834252336', '834252380', '834252565', '834252431

## Explicación del Formato

%m: Representa el mes como un número con dos dígitos (01 a 12).

%d: Representa el día del mes como un número con dos dígitos (01 a 31).

%Y: Representa el año con cuatro dígitos.

%I: Representa la hora en formato de 12 horas (01 a 12).

%M: Representa los minutos.

%p: Representa AM o PM.

En nuestro caso como la fecha tiene los Dias y los Meses sin PADD hay que utilizar la siguiente opción

### pd.to_datetime(salida['FechaHora'], dayfirst=False).dt.strftime('%m/%d/%Y %I:%M %p')


https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

https://pandas.pydata.org/pandas-docs/version/1.0.1/reference/api/pandas.to_datetime.html

https://github.com/pandas-dev/pandas/issues/3341

In Linux "#" is replaced by "-":
%-d, %-H, %-I, %-j, %-m, %-M, %-S, %-U, %-w, %-W, %-y, %-Y
In Windows "-" is replaced by "#":
%#d, %#H, %#I, %#j, %#m, %#M, %#S, %#U, %#w, %#W, %#y, %#Y

#Linux
mydatetime.strftime('%-m/%d/%Y %-I:%M%p')
#Windows
mydatetime.strftime('%#m/%d/%Y %#I:%M%p')
Source: https://msdn.microsoft.com/en-us/library/fe06s4ak.aspx

## Procesar un registro JSON

* https://brightdata.es/blog/procedimientos/parse-json-data-with-python
* https://docs.python.org/3/tutorial/datastructures.html#dictionaries
* https://docs.python.org/3/tutorial/datastructures.html#more-on-lists


In [8]:
# Biscp un registro para probar

reg=2
url = entrada.iloc[reg]['URL']

print(url)
print('Archivo : ',entrada.iloc[reg]['NombreArchivo'])
print('Embarque: ',entrada.iloc[reg]['Embarque'])
print('Fecha   : ',entrada.iloc[reg]['FechaHora'])
print('Tamaño  : ',entrada.iloc[reg]['Tamaño'])
embarque = pd.read_json(url)

http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638501545691039750-834246801.log
Archivo :  638501545691039750-834246801.log
Embarque:  834246801
Fecha   :  2024-05-01 09:02:00
Tamaño  :  1016.0


### Ejemplo de Uso

https://stackoverflow.com/questions/74118717/create-new-dataframe-with-json-objects-from-another-dataframe#:~:text=Solution%20using%20pandas&text=It%20provides%20a%20DataFrame%20object,dataframe%20with%20the%20original%20one



## Procesar El archivo SALIDA identificando los JSON

### PARAMETROS para la CONSULTA.

#### Comienza el proceso

In [9]:
# PARAMETROS SI QUIERO CONSULTAS REGISTROS PUNTUALES --> SOBRE ESCRIBIR REGISTROS

REGISTROS=('834277796', '834277800', '834277808', '834277810', '834277840', '834277842', '834277843', '834277885', '834278471', '834278475', '834278476', '834278484', '834278501', '834278516')


In [11]:
# LLER ARCHIVO JSON
import requests
import json

def procesar_entregas_desde_url(url, modo='imprimir'):
    # Descargar el contenido del archivo JSON desde la URL
    respuesta = requests.get(url)
    respuesta.raise_for_status()  # Esto lanzará una excepción si ocurre un error

    # Leer el contenido JSON
    datos = respuesta.json()    
    resultados = []
    
    # Leer Datos de la Cabecera    
    resultado = f"\nOrigen: {datos['LogisticGroupCode']},  Embarque: {datos['Id']}, Date: {datos['Date']}, Carrier: {datos['CarrierName']}\n"
    resultados.append(resultado)
    # Imprimir la información si el modo es 'imprimir'
    if modo == 'imprimir':
        print(resultado)

    # Procesar y almacenar la información en una lista
    for stop in datos['StopList']:
        # Verificar que la parada tenga entregas
        if stop['OperationType'] == 'Dropped' and stop.get('Deliveries'):
            for delivery in stop['Deliveries']:
                delivery_number = delivery['DeliveryNumber']
                location = stop['LocationName']
                for item in delivery['Items']:
                    material_name = item['MaterialName']
                    batch = item.get('Batch', 'N/A')
                    qty = item['Qty']
                    
                    resultado = f"   DeliveryNumber: {delivery_number}, LocationName: {location}, MaterialName: {material_name}, Batch: {batch}, Qty: {qty}"
                    resultados.append(resultado)

                    # Imprimir la información si el modo es 'imprimir'
                    if modo == 'imprimir':
                        print(resultado)
    
    # Devolver los resultados si el modo es 'devolver'
    if modo == 'devolver':
        return resultados

    # Almacenar los resultados en un archivo si el modo es 'almacenar'
    if modo == 'almacenar':
        with open('resultados.txt', 'a', encoding="utf-8") as archivo_salida:
            for resultado in resultados:
                archivo_salida.write(resultado + '\n')
            archivo_salida.close()
                


# Ejemplo para validar el uso de la función
# url = 'http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638508724764465417-834195574.log'
# procesar_entregas_desde_url(url, modo='imprimir')


#### Explicación
* Leer el archivo JSON: Utilizamos json.load() para cargar el contenido del archivo JSON en una variable datos.
* Procesar las entregas: Iteramos sobre la lista de paradas (StopList). Para cada parada, obtenemos la lista de entregas (Deliveries).
* Extraer y mostrar la información: Para cada entrega, extraemos DeliveryNumber, LocationName, MaterialName, Batch, y Qty, y luego imprimimos esta información en el formato deseado.
* Función versátil: La función procesar_entregas permite imprimir, devolver o almacenar la información según el modo especificado.

##### Ejemplo de uso de la función
procesar_entregas('archivo.json', modo='imprimir')

Utiliza los PARAMETROS ingresados anteriormente

In [12]:

resumen = entrada.loc[entrada["Embarque"].isin(REGISTROS)]
salida = resumen.sort_values(by=['Embarque', 'FechaHora'])

In [13]:
# RUTINA PARA IMPRIMIR SALIDAS por la Terminal

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (REGISTROS):
        print('----------------------------------------------------------------')
        print('Embarque: ',salida.iloc[i]['Embarque'])
        url = salida.iloc[i]['URL']
        print(f'Archivo: ',salida.iloc[i]['FechaHora'],url)
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='imprimir')


----------------------------------------------------------------
Embarque:  834277796
Archivo:  2024-05-17 16:04:00 http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638515622639871561-834277796.log

Origen: SAPP4S,  Embarque: 834277796, Date: 2024-05-20T16:58:00Z, Carrier: ZARCAM S.A.

   DeliveryNumber: 8480744534, LocationName: AGRO GESTION DEL LITORAL SA, MaterialName: SCENIC FS80 4X5L BOT AR, Batch: None, Qty: 16.0
   DeliveryNumber: 8480744534, LocationName: AGRO GESTION DEL LITORAL SA, MaterialName: HUSSAR OD PLUS PACK KK 1X1PC COP AR, Batch: LA40006612, Qty: 16.0
   DeliveryNumber: 8480743522, LocationName: ENRIQUE M BAYA CASAL SA, MaterialName: HUSSAR PLUS II PACK KK 1X1PCE COP AR, Batch: LA40006763, Qty: 50.0
   DeliveryNumber: 8480743522, LocationName: ENRIQUE M BAYA CASAL SA, MaterialName: SCENIC FS80 4X5L BOT AR, Batch: LA40006772, Qty: 400.0
   DeliveryNumber: 8480743522, LocationName: ENRIQUE M BAYA CASAL SA, MaterialName: CHUCARO FS330 4X5L BOT AR, Batch: None, Qty: 80.0
 

In [44]:
# RUTINA PARA GRABAR en Archivo las SALIDAS 

with open('resultados.txt', 'w', encoding="utf-8") as f:
    f.write(f"PROCESO DE INTEFACES desde resultado {fecha_inicio}" + '\n')
    f.close()

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (REGISTROS):
        with open('resultados.txt', 'a', encoding="utf-8") as f:
            f.write('\n----------------------------------------------------------------\n')
            f.write(f"Embarque: '{salida.iloc[i]['Embarque']}")
            url = salida.iloc[i]['URL']
            f.write(f"Archivo:  {salida.iloc[i]['FechaHora']} - {url}")
            f.close()
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='almacenar')

### Otras Alternativas para Analizar JSON Complejos

https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/python-jsonpath/#:~:text=Para%20que%20Python%20pueda%20leer,%E2%80%9D%20o%20%E2%80%9Cload()%E2%80%9D.



In [14]:
salida.head(10)

,NombreArchivo,URL,FechaHora,Tamaño,Embarque
3795,638513616518159531-834271072.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 08:20:00,1154.0,834271072
3796,638513624739445624-834271963.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 08:34:00,909.0,834271963
3797,638513624741367484-834271964.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 08:34:00,909.0,834271964
3798,638513635564677811-834271969.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 08:52:00,1037.0,834271969
3799,638513635566049432-834271968.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 08:52:00,908.0,834271968
3800,638513638797574491-834270573.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 08:57:00,919.0,834270573
3801,638513652771919900-834272224.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 09:21:00,1035.0,834272224
3802,638513662294083761-834272227.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 09:37:00,1036.0,834272227
3803,638513662576588649-834270202.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 09:37:00,3964.0,834270202
3804,638513667202650110-834270202.log,http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...,2024-05-15 09:45:00,3964.0,834270202


In [15]:
# EXPORTAR los datos a EXCEL 
salida.to_excel("./salida.xlsx")

## Comparar con los Datos que están en SQL SERVER

In [16]:
import pyodbc

SERVER = '213.134.40.73,9595'
DATABASE = 'seamtrack'
USERNAME = 'eduardo.ettlin@vigiloo.com.ar'
PASSWORD = 'Aladelta10$'
OPTIONS= 'encrypt=no'

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};{OPTIONS}'
print(connectionString)
conn = pyodbc.connect(connectionString) 

DRIVER={ODBC Driver 18 for SQL Server};SERVER=213.134.40.73,9595;DATABASE=seamtrack;UID=eduardo.ettlin@vigiloo.com.ar;PWD=Aladelta10$;encrypt=no


In [20]:
# PARAMETROS

SQL_QUERY = f"""
SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN ({REGISTROS} )
ORDER BY A.RefId,D.RefId, D.[DestinationCode],E.[MaterialCode];
"""

In [21]:
print(SQL_QUERY)


SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN ('834271072', '834271963', '834271964', '834271969', '834271968', '834270573', '834272224', '834272227', '834270202', '834269886', '834272363', '834270126', '834270125', '834272369', '834270016', '834272505', '834272570', '834269911', '834269804', '834272871', '834272879', '834273008', '834273009', '834273065', '834273292', '834272875', '834272989', '834272877', '834273050', '834273061', '834273062', '834272874', '834272988', '834272876', '834195574', '

In [22]:
cursor = conn.cursor()
cursor.execute(SQL_QUERY)

In [23]:
records = cursor.fetchall()
for r in records:
    print(f"{r.EMBARQUE}\t{r.DELIVERY}\t{r.FECHA}\t{r.DestinationCode}\t{r.DestinationName}\t{r.MaterialName}\t{r.Batch}\t{r.QTY}\t{r.LogisticGroupId}")

834195574	5011454954	0001-01-01 00:00:00	0001887492	AGRO GESTION DEL LITORAL SA	RUP CONTROLMAX,AR,64x15KG BOX	None	  1020	75247427-34FD-41A3-B7E0-F0B8312EADA8
834195574	5011454954	0001-01-01 00:00:00	0001887492	AGRO GESTION DEL LITORAL SA	RUP TOP,AR,48x20L BT MON301515	None	    20	75247427-34FD-41A3-B7E0-F0B8312EADA8
834195574	5011454954	0001-01-01 00:00:00	0001887492	AGRO GESTION DEL LITORAL SA	RUP FG W,AR,64X15KG BOX MON76818	None	   180	75247427-34FD-41A3-B7E0-F0B8312EADA8
834195574	5011454957	0001-01-01 00:00:00	0003398927	EL MALACATE SA	HARNESS,AR,48x20L BT MON58415	None	   120	75247427-34FD-41A3-B7E0-F0B8312EADA8
834195574	5011454957	0001-01-01 00:00:00	0003398927	EL MALACATE SA	RUP CONTROLMAX,AR,64x15KG BOX	None	   180	75247427-34FD-41A3-B7E0-F0B8312EADA8
834195574	5011454957	0001-01-01 00:00:00	0003398927	EL MALACATE SA	RUP TOP,AR,48x20L BT MON301515	None	   120	75247427-34FD-41A3-B7E0-F0B8312EADA8
834195574	5011456282	0001-01-01 00:00:00	0003398927	EL MALACATE SA	HARNESS,AR,48

In [24]:
df = pd.DataFrame((tuple(t) for t in records)) 
df.columns = ['EMBARQUE','DELIVERY','FECHA','PEDIDO','DestinationCode','DestinationName','MaterialCode','MaterialName','Batch','QTY','LogisticGroupId']

In [25]:
# Adecuar el Grupo Logístico
df["LogisticGroupId"] = df["LogisticGroupId"].replace(
    { "75247427-34FD-41A3-B7E0-F0B8312EADA8": "Monsanto", "F21EE1F9-4823-42B4-AD40-8C81F2847818": "BAYER" })

In [26]:
df.head(20)

,EMBARQUE,DELIVERY,FECHA,PEDIDO,DestinationCode,DestinationName,MaterialCode,MaterialName,Batch,QTY,LogisticGroupId
0,834195574,5011454954,0001-01-01 00:00:00,0828992862,0001887492,AGRO GESTION DEL LITORAL SA,12331311,"RUP CONTROLMAX,AR,64x15KG BOX",None,1020,Monsanto
1,834195574,5011454954,0001-01-01 00:00:00,0828992862,0001887492,AGRO GESTION DEL LITORAL SA,30146700,"RUP TOP,AR,48x20L BT MON301515",None,20,Monsanto
2,834195574,5011454954,0001-01-01 00:00:00,0828992862,0001887492,AGRO GESTION DEL LITORAL SA,30248923,"RUP FG W,AR,64X15KG BOX MON76818",None,180,Monsanto
3,834195574,5011454957,0001-01-01 00:00:00,0829000428,0003398927,EL MALACATE SA,10434277,"HARNESS,AR,48x20L BT MON58415",None,120,Monsanto
4,834195574,5011454957,0001-01-01 00:00:00,0829000428,0003398927,EL MALACATE SA,12331311,"RUP CONTROLMAX,AR,64x15KG BOX",None,180,Monsanto
5,834195574,5011454957,0001-01-01 00:00:00,0829000428,0003398927,EL MALACATE SA,30146700,"RUP TOP,AR,48x20L BT MON301515",None,120,Monsanto
6,834195574,5011456282,0001-01-01 00:00:00,0828999362,0003398927,EL MALACATE SA,10434277,"HARNESS,AR,48x20L BT MON58415",None,120,Monsanto
7,834195574,5011456297,0001-01-01 00:00:00,0828994377,0001887492,AGRO GESTION DEL LITORAL SA,12331311,"RUP CONTROLMAX,AR,64x15KG BOX",None,4005,Monsanto
8,834195574,5011456298,0001-01-01 00:00:00,0828994390,0001887492,AGRO GESTION DEL LITORAL SA,12331311,"RUP CONTROLMAX,AR,64x15KG BOX",None,1020,Monsanto
9,834195574,5011456301,0001-01-01 00:00:00,0828999388,0003398927,EL MALACATE SA,12331311,"RUP CONTROLMAX,AR,64x15KG BOX",None,180,Monsanto


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   EMBARQUE         197 non-null    object
 1   DELIVERY         197 non-null    object
 2   FECHA            197 non-null    object
 3   PEDIDO           197 non-null    object
 4   DestinationCode  197 non-null    object
 5   DestinationName  197 non-null    object
 6   MaterialCode     197 non-null    object
 7   MaterialName     197 non-null    object
 8   Batch            101 non-null    object
 9   QTY              197 non-null    object
 10  LogisticGroupId  197 non-null    object
dtypes: object(11)
memory usage: 17.1+ KB


In [27]:
df.to_excel("./SQLServer.xlsx")